In [257]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

In [258]:
df = pd.read_csv('compas-scores-two-years.csv')
df = df[data.race.isin(['Caucasian', 'African-American'])]
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1
8,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,Caucasian,...,1,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0


### Tratando os dados

Assim como a análise da ProPublica 

In [259]:
column_names = ['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count',
                'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid',
                'c_jail_in', 'c_jail_out']

In [260]:
df = df[df['days_b_screening_arrest'] <= 30]
df = df[df['days_b_screening_arrest'] >= -30]
df = df[df['is_recid'] != -1]
df = df[df['c_charge_degree'] != 'O']
df = df[df['score_text'] != 'N/A']
df[column_names].head()

,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out
1,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53
2,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04
6,41,F,Caucasian,25 - 45,Medium,Male,14,-1.0,6,1,1,2014-02-18 05:08:24,2014-02-24 12:18:30
8,39,M,Caucasian,25 - 45,Low,Female,0,-1.0,1,0,0,2014-03-15 05:35:34,2014-03-18 04:28:46
10,27,F,Caucasian,25 - 45,Low,Male,0,-1.0,4,0,0,2013-11-25 06:31:06,2013-11-26 08:26:57


In [261]:
sex_dict = dict(enumerate(df['sex'].astype('category').cat.categories))
print(sex_dict)
df['c_sex'] = df['sex'].astype('category').cat.codes

{0: 'Female', 1: 'Male'}


In [262]:
age_dict = dict(enumerate(df['age_cat'].astype('category').cat.categories))
print(age_dict)
df['c_age_cat'] = df['age_cat'].astype('category').cat.codes

{0: '25 - 45', 1: 'Greater than 45', 2: 'Less than 25'}


In [263]:
charge_dict = dict(enumerate(df['c_charge_degree'].astype('category').cat.categories))
print(charge_dict)
df['c_c_charge_degree'] = df['c_charge_degree'].astype('category').cat.codes

{0: 'F', 1: 'M'}


In [264]:
race_dict = dict(enumerate(df['race'].astype('category').cat.categories))
print(race_dict)
df['c_race'] = df['race'].astype('category').cat.codes

{0: 'African-American', 1: 'Caucasian'}


In [265]:
score_dict = dict(enumerate(df['score_text'].astype('category').cat.categories))
print(score_dict)
df['c_score_text'] = df['score_text'].astype('category').cat.codes

{0: 'High', 1: 'Low', 2: 'Medium'}


In [266]:
# df['c_jail_in'].astype('datetime64').astype('int64')
df['length_of_stay'] =  df['c_jail_out'].astype('datetime64').astype('int64') - df['c_jail_in'].astype('datetime64').astype('int64')

In [267]:
c_column_names = ['age', 'c_c_charge_degree', 'c_race', 'c_age_cat', 'c_score_text', 'c_sex', 'priors_count',
                'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid']
# c_column_names = ['age', 'c_c_charge_degree', 'c_race', 'c_age_cat', 'c_score_text', 'c_sex', 'priors_count',
#                 'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid', 'length_of_stay']
df[c_column_names].head()

,age,c_c_charge_degree,c_race,c_age_cat,c_score_text,c_sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid
1,34,0,0,0,1,1,0,-1.0,3,1,1
2,24,0,0,2,1,1,4,-1.0,4,1,1
6,41,0,1,0,2,1,14,-1.0,6,1,1
8,39,1,1,0,1,0,0,-1.0,1,0,0
10,27,0,1,0,1,1,0,-1.0,4,0,0


In [268]:
df2 = df[c_column_names]
# X = df2.loc[:, df2.columns != 'two_year_recid']
X = df2.loc[:, df2.columns != 'decile_score']
X.head()

,age,c_c_charge_degree,c_race,c_age_cat,c_score_text,c_sex,priors_count,days_b_screening_arrest,is_recid,two_year_recid
1,34,0,0,0,1,1,0,-1.0,1,1
2,24,0,0,2,1,1,4,-1.0,1,1
6,41,0,1,0,2,1,14,-1.0,1,1
8,39,1,1,0,1,0,0,-1.0,0,0
10,27,0,1,0,1,1,0,-1.0,0,0


In [269]:
# y = df2['two_year_recid']
y = df2['decile_score']
y.head()

1     3
2     4
6     6
8     1
10    4
Name: decile_score, dtype: int64

In [270]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [271]:
logistic_regression_cv = LogisticRegressionCV(
    cv=5, max_iter=10000, n_jobs=2, verbose=1, multi_class="auto").fit(
    X_train, y_train)


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:   34.6s finished


In [272]:
y_pred = logistic_regression_cv.predict(X_test)

In [273]:
prfs = precision_recall_fscore_support(y_test, y_pred)
prfs[:-1]
# pd.DataFrame(prfs[:-1],index=["Precisão", "Revocação", "F1"])

(array([0.63092269, 0.42666667, 0.44      , 0.41584158, 0.45081967,
        0.40625   , 0.37795276, 0.38686131, 0.34883721, 0.46666667]),
 array([0.8815331 , 0.45070423, 0.06145251, 0.48554913, 0.71895425,
        0.17808219, 0.35294118, 0.48623853, 0.42056075, 0.17283951]),
 array([0.73546512, 0.43835616, 0.10784314, 0.448     , 0.55415617,
        0.24761905, 0.36501901, 0.43089431, 0.38135593, 0.25225225]))

### Cálculo das métricas

In [313]:
df_temp = X_test
df_temp['pred_decile_score'] = y_pred
df_temp['pred_decile_score'][89]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


7

In [319]:
df_metrics = X_test.copy()
df_metrics['pred_decile_score'] = y_pred

In [343]:
# matrix = threshold x race_code

for threshold in range(0, 10):
    for race_code in range(0, 2):
        num = df_metrics[df_metrics["pred_decile_score"] > threshold & df_metrics["c_race"] == race_code & df_metrics["two_year_recid"] == 0].size
        dem = df_metrics[df_metrics["c_race"] == race_code & df_metrics["two_year_recid"] == 0].size
        print(num/dem)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [282]:
df_resposta = df
df_resposta['pred_decile_score'] = y_pred

ValueError: Length of values does not match length of index